In [20]:
# !pip install langchain_community

In [21]:
from langchain_community.llms import Ollama

In [22]:
# llm = Ollama(model="llama2")
llm = Ollama(model="sachin2505/accountant")
llm.invoke("The first man on the moon was ...")

"\nExcuse me, but as a financial assistant, I don't have personal opinions or beliefs. My purpose is to assist you in organizing and categorizing your financial information. Can I help you with that? Please provide me with the necessary bank transaction data, and I will be happy to classify the expenses and income for you."

In [23]:
llm.invoke("Can you add an appropriate category next to each of the following expenses. Respond with a list of categories separated by commas. For example, Spotify AB by Adyen - \
Entertainment, Beta Boulders Ams Amsterdam Nld - Sports, etc.: \
Taxi Utrecht, Ministerie van Justitie en Veiligheid, Etos AMSTERDAM NLD, Bistro Bar Amsterdam")

"\nOf course! I'd be happy to help you classify your expenses. Based on the information provided, here are the categories I would suggest for each transaction:\n\nTaxi Utrecht - Transportation\nMinisterie van Justitie en Veiligheid - Government\nEtos AMSTERDAM NLD - Grocery/Food\nBistro Bar Amsterdam - Dining/Entertainment\n\nI hope this helps! Let me know if you have any other questions."

### Read transaction data

In [24]:
# Read the transactions_2022_2023.csv file 
import pandas as pd
df = pd.read_csv("transactions_2022_2023.csv")
df.head()

,Date,Name / Description,Expense/Income,Amount (EUR)
0,2023-12-30,Belastingdienst,Expense,9.96
1,2023-12-30,Tesco Breda,Expense,17.53
2,2023-12-30,Monthly Appartment Rent,Expense,451.00
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46
4,2023-12-29,Selling Paintings,Income,13.63


In [25]:
# Get unique transactions in the Name / Description column
unique_transactions = df["Name / Description"].unique()
len(unique_transactions)

23

In [26]:
unique_transactions[1:10]

array(['Tesco Breda', 'Monthly Appartment Rent',
       'Vishandel Sier Amsterdam', 'Selling Paintings',
       'Spotify Ab By Adyen', 'Tk Maxx Amsterdam Da', 'Consulting',
       'Aidsfonds', 'Tls Bv Inz Ov-Chipkaart'], dtype=object)

### Categorise bank transactions with Llama2

In [27]:
# Get index list
#https://stackoverflow.com/questions/47518609/for-loop-range-and-interval-how-to-include-last-step
def hop(start, stop, step):
    for i in range(start, stop, step):
        yield i
    yield stop

index_list = list(hop(0, len(unique_transactions), 30))
index_list

[0, 23]

In [28]:
def categorize_transactions(transaction_names, llm):
    response = llm.invoke("Can you add an appropriate category to the following expenses. For example: Spotify AB by Adyen - Entertainment, Beta Boulders Ams Amsterdam Nld - Sport, etc.. Categories should be less than 4 words. " + transaction_names)
    response = response.split('\n')

    print(response)

    # Put in dataframe
    categories_df = pd.DataFrame({'Transaction vs category': response})
    categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.split(' - ', expand=True)
    
    return categories_df

In [29]:
# Test out the function
categorize_transactions('ISS Catering Services De Meern, Taxi Utrecht, Etos AMSTERDAM NLD, Bistro Bar Amsterdam',
                        llm)

['Of course! Here are the categories for each of the expenses you provided:', '', '1. Spotify AB by Adyen - Entertainment', '2. ISS Catering Services De Meern - Food & Beverage', '3. Taxi Utrecht - Transportation', '4. Etos AMSTERDAM NLD - Retail', '5. Bistro Bar Amsterdam - Food & Beverage']


,Transaction vs category,Transaction,Category
0,Of course! Here are the categories for each of...,Of course! Here are the categories for each of...,None
1,,,None
2,1. Spotify AB by Adyen - Entertainment,1. Spotify AB by Adyen,Entertainment
3,2. ISS Catering Services De Meern - Food & Bev...,2. ISS Catering Services De Meern,Food & Beverage
4,3. Taxi Utrecht - Transportation,3. Taxi Utrecht,Transportation
5,4. Etos AMSTERDAM NLD - Retail,4. Etos AMSTERDAM NLD,Retail
6,5. Bistro Bar Amsterdam - Food & Beverage,5. Bistro Bar Amsterdam,Food & Beverage


In [30]:
# Intialise the categories_df_all dataframe
categories_df_all = pd.DataFrame()

# Loop through the index_list
for i in range(0, len(index_list)-1):
    transaction_names = unique_transactions[index_list[i]:index_list[i+1]]
    transaction_names = ','.join(transaction_names)

    categories_df = categorize_transactions(transaction_names, llm)
    categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)

['Of course! Here are the categories I would assign to each of the expenses you listed:', '', '1. Spotify AB by Adyen - Entertainment', '2. Belastingdienst - Utilities', '3. Tesco Breda - Groceries', '4. Monthly Appartment Rent - Housing', '5. Vishandel Sier Amsterdam - Groceries', '6. Selling Paintings - Miscellaneous Income', '7. Spotify Ab By Adyen - Music Streaming', '8. Tk Maxx Amsterdam Da - Retail', '9. Consulting - Professional Services', '10. Aidsfonds - Donations', '11. Tls Bv Inz Ov-Chipkaart - Transportation', '12. Etos Amsterdam - Groceries', '13. Beta Boulders Ams Amsterdam - Fitness', '14. Salary - Income', '15. Bouldermuur Bv Amsterdam - Professional Services', '16. Birtat Restaurant Amsterdam - Dining Out', '17. Freelancing - Professional Services', '18. Tikkie - Financial Services', '19. Blogging - Miscellaneous Income', '20. Taxi Utrecht - Transportation', '21. Apple Services - Technology', '22. Amazon Lux - Retail', '23. Classpass* Monthly - Fitness', '24. Audible U

In [31]:
categories_df_all

,Transaction vs category,Transaction,Category
0,Of course! Here are the categories I would ass...,Of course! Here are the categories I would ass...,None
1,,,None
2,1. Spotify AB by Adyen - Entertainment,1. Spotify AB by Adyen,Entertainment
3,2. Belastingdienst - Utilities,2. Belastingdienst,Utilities
4,3. Tesco Breda - Groceries,3. Tesco Breda,Groceries
5,4. Monthly Appartment Rent - Housing,4. Monthly Appartment Rent,Housing
6,5. Vishandel Sier Amsterdam - Groceries,5. Vishandel Sier Amsterdam,Groceries
7,6. Selling Paintings - Miscellaneous Income,6. Selling Paintings,Miscellaneous Income
8,7. Spotify Ab By Adyen - Music Streaming,7. Spotify Ab By Adyen,Music Streaming
9,8. Tk Maxx Amsterdam Da - Retail,8. Tk Maxx Amsterdam Da,Retail


In [32]:
# categories_df_all.to_csv("categories_df_all.csv", index=False)

In [33]:
# Get unique categories in categories_df_all
unique_categories = categories_df_all["Category"].unique()
unique_categories

array([None, 'Entertainment', 'Utilities', 'Groceries', 'Housing',
       'Miscellaneous Income', 'Music Streaming', 'Retail',
       'Professional Services', 'Donations', 'Transportation', 'Fitness',
       'Income', 'Dining Out', 'Financial Services', 'Technology'],
      dtype=object)

In [34]:
# Drop NA values
categories_df_all = categories_df_all.dropna()

# If category contains "Food", then categorise as "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Food"), 'Category'] = "Food and Drinks"
# If category contains "Clothing", then categorise as "Clothing"
categories_df_all.loc[categories_df_all['Category'].str.contains("Clothing"), 'Category'] = "Clothing"
# If category contains "Services", then categorise as "Services"
categories_df_all.loc[categories_df_all['Category'].str.contains("Services"), 'Category'] = "Services"
# If category contains "Health" or "Wellness", then categorise as "Health and Wellness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Health|Wellness"), 'Category'] = "Health and Wellness"
# If category contains "Sport", then categorise as "Sport
#  and Fitness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Sport"), 'Category'] = "Sport and Fitness"
# If category contains "Travel", then categorise as "Travel"
categories_df_all.loc[categories_df_all['Category'].str.contains("Travel"), 'Category'] = "Travel"

In [35]:
# Remove the numbering eg "1. " from Transaction column
categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')
categories_df_all

/tmp/ipykernel_433369/2440992137.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')


,Transaction vs category,Transaction,Category
2,1. Spotify AB by Adyen - Entertainment,1. Spotify AB by Adyen,Entertainment
3,2. Belastingdienst - Utilities,2. Belastingdienst,Utilities
4,3. Tesco Breda - Groceries,3. Tesco Breda,Groceries
5,4. Monthly Appartment Rent - Housing,4. Monthly Appartment Rent,Housing
6,5. Vishandel Sier Amsterdam - Groceries,5. Vishandel Sier Amsterdam,Groceries
7,6. Selling Paintings - Miscellaneous Income,6. Selling Paintings,Miscellaneous Income
8,7. Spotify Ab By Adyen - Music Streaming,7. Spotify Ab By Adyen,Music Streaming
9,8. Tk Maxx Amsterdam Da - Retail,8. Tk Maxx Amsterdam Da,Retail
10,9. Consulting - Professional Services,9. Consulting,Services
11,10. Aidsfonds - Donations,10. Aidsfonds,Donations


In [36]:
# Merge the categories_df_all with the transactions_2022_2023.csv dataframe (df)
df = pd.read_csv("transactions_2022_2023.csv")
df.loc[df['Name / Description'].str.contains("Spotify"), 'Name / Description'] = "Spotify Ab By Adyen"
df = pd.merge(df, categories_df_all, left_on='Name / Description', right_on='Transaction', how='left')
df

,Date,Name / Description,Expense/Income,Amount (EUR),Transaction vs category,Transaction,Category
0,2023-12-30,Belastingdienst,Expense,9.96,NaN,NaN,NaN
1,2023-12-30,Tesco Breda,Expense,17.53,NaN,NaN,NaN
2,2023-12-30,Monthly Appartment Rent,Expense,451.00,NaN,NaN,NaN
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46,NaN,NaN,NaN
4,2023-12-29,Selling Paintings,Income,13.63,NaN,NaN,NaN
5,2023-12-29,Spotify Ab By Adyen,Expense,12.19,NaN,NaN,NaN
6,2023-12-23,Tk Maxx Amsterdam Da,Expense,27.08,NaN,NaN,NaN
7,2023-12-22,Consulting,Income,541.57,NaN,NaN,NaN
8,2023-12-22,Aidsfonds,Expense,10.70,NaN,NaN,NaN
9,2023-12-20,Consulting,Income,2641.93,NaN,NaN,NaN


In [37]:
df.to_csv("transactions_2022_2023_categorized.csv", index=False)